# MySQL 연결

In [25]:
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='jsp', passwd='12345', db='djit')
cur = conn.cursor()
cur.execute("use djit")
# cur.execute('drop table scrapying')
cur.execute("create table scrapying(title varchar(100),content varchar(1000))") 
#cur.execute("insert into scrapying values(1,'홍길동', 30)")
cur.execute("commit")

# cur.execute("select * from scrapying")
# print(cur.fetchone())
cur.close()
conn.close()

In [26]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql
import re

conn = pymysql.connect(host='127.0.0.1', user='jsp', passwd='12345', db='djit', charset='utf8')
cur = conn.cursor() 
cur.execute('use djit')

random.seed(datetime.datetime.now())

def store(title, content):
    cur.execute('insert into scrapying (title, content) values ("%s", "%s")',(title, content))
    cur.connection.commit()

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org' + articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    print(title)
    print(content)
    store(title,content)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
try:
    while len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        print(newArticle)
        links = getLinks(newArticle)
finally:
    cur.close()
    conn.close()

Kevin Bacon


/wiki/Robert_Duvall
Robert Duvall


/wiki/Ron_Howard
Ron Howard


/wiki/Phil_Lord_and_Christopher_Miller
Phil Lord and Christopher Miller


/wiki/Andrew_Stanton
Andrew Stanton
Andrew Ayers Stanton (born December 3, 1965) is an American animator, storyboard artist, film director, screenwriter, producer and voice actor based at Pixar, which he joined in 1990.[2] His film work includes co-writing and co-directing Pixar's A Bug's Life (1998), directing Finding Nemo (2003)[3] and the sequel Finding Dory (2016), WALL-E (2008), and the live-action film, Disney's John Carter (2012), and co-writing all four Toy Story films (1995–2019) and Monsters, Inc. (2001).

/wiki/Adrian_Molina
Adrian Molina
Adrian Molina (born August 23, 1985) is an American animator, storyboard artist, screenwriter, director, and lyricist. He has been at Pixar since 2007, where he started as a 2D animator on Ratatouille. He later moved on to be a storyboard artist, working on Toy Story 3 and Monsters Univers

DataError: (1406, "Data too long for column 'content' at row 1")